In [1]:
pip install selenium

     |████████████████████████████████| 904 kB 6.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np

In [3]:
from gurobi import *

In [33]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import pandas as pd
import csv


# Get variable names
with open('AirbnbTrain.csv') as csvFile:
    reader = csv.reader(csvFile)
    features = next(reader) ## stores all the listings 

# Load data
listing_data = genfromtxt('AirbnbTrain.csv', delimiter=',',skip_header = 1)

### FOR TESTING: 
# Load data
testing_data = genfromtxt('AirbnbTest.csv', delimiter=',',skip_header = 1)
X_test = testing_data[:,0:12]
y_true = testing_data[:,[12]]
# get dimensions of data
d = listing_data.shape[0]
n = listing_data.shape[1]
t = testing_data.shape[0]

### Model 1 

In [34]:
mod1 = Model()
y = np.array(listing_data[:,12])
x = np.array(listing_data[:,:12])

## Adding Variables
b = mod1.addVars(n)
#w = mod1.addVars(n, vtype = GRB.BINARY)
Error = mod1.addVars(d)

## Adding Constraints 
for i in range(d):
    mod1.addConstr(Error[i] >= y[i] - sum(b[j]*x[i][j] for j in range(n-1)))
    mod1.addConstr(Error[i] >= -y[i] + sum(b[j]*x[i][j] for j in range(n-1)))  
    
## Setting Objective Function    
mod1.setObjective((1/d)*sum(Error[i] for i in range(d)), GRB.MINIMIZE)

In [35]:
mod1.update()
mod1.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 3400 rows, 1713 columns and 41372 nonzeros
Model fingerprint: 0xfda29b00
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 0 rows and 1 columns
Presolve time: 0.04s
Presolved: 3400 rows, 1712 columns, 41372 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 3.967e+04
 Factor NZ  : 4.315e+04 (roughly 2 MBytes of memory)
 Factor Ops : 5.492e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.12283752e+03  0.00000000e+00  1.36e+03 0.00e+00  2.67e+01

In [36]:
beta = [b[i].X for i in range(n)]
y_pred = []
for j in range(t):
            y_pred.append(sum(X_test[j][i]*beta[i] for i in range(n-1)))

In [37]:
from sklearn.metrics import mean_absolute_error

In [38]:
print('Prediction Error ($/night):', mean_absolute_error(y_true, y_pred))

Prediction Error ($/night): 35.60453503037782


### Model 2

In [39]:
mod2 = Model()
y = np.array(listing_data[:,12])
x = np.array(listing_data[:,:12])

## Adding Variables
b = mod2.addVars(n)
w = mod2.addVars(n, vtype = GRB.BINARY)
Error = mod2.addVars(d)

## Adding Constraints 
for i in range(d):
    mod2.addConstr(Error[i] >= y[i] - sum(b[j]*w[j]*x[i][j] for j in range(n-1)))
    mod2.addConstr(Error[i] >= -y[i] + sum(b[j]*w[j]*x[i][j] for j in range(n-1))) 
for j in range(n-1):
    mod2.addConstr(b[j]>= w[j] )
    
mod2.addConstr(quicksum(w[j] for j in range(n-1)) <=3)
   
## Setting Objective Function    
mod2.setObjective((1/d)*sum(Error[i] for i in range(d)), GRB.MINIMIZE)

In [40]:
mod2.update()
mod2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 13 rows, 1726 columns and 36 nonzeros
Model fingerprint: 0xca1df099
Model has 3400 quadratic constraints
Variable types: 1713 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e-01, 5e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
  QRHS range       [1e+01, 2e+03]
Found heuristic solution: objective 2.000000e+09
Presolve added 824 rows and 408 columns
Presolve time: 0.15s
Presolved: 3433 rows, 2170 columns, 32982 nonzeros
Presolved model has 24 SOS constraint(s)
Variable types: 2146 continuous, 24 integer (24 binary)

Root relaxation: objective 3.642625e+01, 2581 iterations, 0.24 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Ob

In [41]:
beta = [b[i].X for i in range(n)]
y_pred = []
for j in range(t):
            y_pred.append(sum(X_test[j][i]*beta[i] for i in range(n-1)))
        
#######################PART A###########################################

print('a)')    
for i in [2,3,5]:
    print('    ',features[i],':',b[i].X)
    
#######################PART B###########################################

print('b)')  
print('     New Prediction Error ($/night):', mean_absolute_error(y_true, y_pred))

a)
     Entire home : 52.0
     accommodates : 14.0
     bedrooms : 32.0
b)
     New Prediction Error ($/night): 37.73676680972818


### Model 3

In [42]:
mod3 = Model()
y = np.array(listing_data[:,12])
x = np.array(listing_data[:,:12])

## Adding Variables
b = mod3.addVars(n)
w = mod3.addVars(n, vtype = GRB.BINARY)
Error = mod3.addVars(d)

## Adding Constraints 
for i in range(d):
    mod3.addConstr(Error[i] >= y[i] - sum(b[j]*w[j]*x[i][j] for j in range(n-1)))
    mod3.addConstr(Error[i] >= -y[i] + sum(b[j]*w[j]*x[i][j] for j in range(n-1))) 
for j in range(n-1):
    mod3.addConstr(b[j]>= w[j])

mod3.addConstr(quicksum(w[j] for j in range(n-1)) <=3)
mod3.addConstr(w[6] == 1)
   
## Setting Objective Function    
mod3.setObjective((1/d)*sum(Error[i] for i in range(d)), GRB.MINIMIZE)

In [43]:
mod3.update()
mod3.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 14 rows, 1726 columns and 37 nonzeros
Model fingerprint: 0x9ec7ed6f
Model has 3400 quadratic constraints
Variable types: 1713 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e-01, 5e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
  QRHS range       [1e+01, 2e+03]
Presolve added 822 rows and 407 columns
Presolve time: 0.15s
Presolved: 3430 rows, 2166 columns, 32974 nonzeros
Presolved model has 22 SOS constraint(s)
Variable types: 2144 continuous, 22 integer (22 binary)
Found heuristic solution: objective 60.5111765

Root relaxation: objective 3.645095e+01, 2565 iterations, 0.40 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj 

In [45]:
#######################PART A##########################################

print('a)')    
for i in [2,5,6]:
    print('    ',features[i],':',b[i].X)

#######################PART B##########################################

print('b)')
print('     Accomodates was dropped between Model 2 and 3. This variable had the lowest beta coefficient meaning that it has the least influence on price in Model 2, so when beds was fixed, the model chose to drop it in order to keep the remaining "more significant" coefficients.') 

#######################PART C###########################################

beta = [b[i].X for i in range(n)]
y_pred = []
for j in range(t):
            y_pred.append(sum(X_test[j][i]*beta[i] for i in range(n-1)))
print('\nc)')
print('     New Prediction Error ($/night):', mean_absolute_error(y_true, y_pred))
print('     It is expected for the prediction error to increase due to the presence of additional constraints on the model.')

a)
     Entire home : 67.875
     bedrooms : 47.37499999999999
     beds : 12.125000000000002
b)
     Accomodates was dropped between Model 2 and 3. This variable had the lowest beta coefficient meaning that it has the least influence on price in Model 2, so when beds was fixed, the model chose to drop it in order to keep the remaining "more significant" coefficients.

c)
     New Prediction Error ($/night): 38.59960658082976
     It is expected for the prediction error to increase due to the presence of additional constraints on the model.
